|课程名称：机器学习|学生姓名：邓力予|学生学号：20201910442|
|-|-|-|
|实验名称：机器学习实验三|
|学院：数学与统计学院|专业：数据科学与大数据技术|年级：2020级|

# 准备工作

## 导入包

In [1]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from pandas import read_csv
import numpy as np
import joblib
import copy
import scipy.spatial.distance as dist

## 读取数据

### 从文件读取数据

In [2]:
#使用int16降低精度防止后续计算内存溢出
data_train=read_csv("../fashion-mnist_train.csv",dtype=np.int16)
data_test=read_csv("../fashion-mnist_test.csv",dtype=np.int16)

### 拆分数据集

- 训练集

In [3]:
data_train_x = data_train.iloc[:, 1:]
data_train_y = data_train["label"]

- 测试集

In [4]:
data_test_x = data_test.iloc[:, 1:]
data_test_y = data_test["label"]

# 使用scikit-learn

## 构造模型

In [5]:
model=SVC(gamma='scale', C=1.0, decision_function_shape='ovr', kernel='rbf')

## 训练模型

In [6]:
try:
    model=joblib.load('model.pkl')
except FileNotFoundError:
    model.fit(data_train_x,data_train_y)
    joblib.dump(model, 'model.pkl')

## 测试

- 使用测试集进行预测

In [7]:
try:
    with open("sklearn_pre.txt","r+") as file:
        data_test_y_pre=np.array(eval(file.read()))
except FileNotFoundError:
    data_test_y_pre = model.predict(data_test_x)
    with open("sklearn_pre.txt","w+") as file:
        file.write(str(tuple(data_test_y_pre)))

- 计算正确率

In [8]:
score_sklearn = accuracy_score(data_test_y, data_test_y_pre)
print(score_sklearn)

0.8921


# 不使用scikit-learn

## 构造模型

- 训练类

In [9]:
from __future__ import division
class SvmTrain(object):
    def __init__(self, C, max_iters, tol, kernel, seta, gamma, degree):
        self.C = C
        self.max_iters = max_iters
        self.tol = tol
        self.kernel = kernel
        self.seta = seta
        self.gamma = gamma
        self.degree = degree
        
    def random_idx(self, i, n):
        j = i
        while j == i:
            j = np.random.randint(0, n)
        return j
    
    def find_bounds(self, a, i, j):
        if self.y[i] == self.y[j]:
            L = max(0, a[i] + a[j] - self.C)
            H = min(a[i] + a[j], self.C)
        else:
            L = max(0, a[j] - a[i])
            H = min(self.C, self.C + a[j] - a[j])
        return L, H
    
    def clip(self, a, L, H):
        if a > H:
            a = H
        elif a < L:
            a = L
        else:
            a = a
        return a
    
    def linear_kernel(self, X1, X2):
        res = X1.dot(X2.T)
        return res
    
    def poly_kernel(self, X1, X2):
        res = (self.seta + self.gamma*X1.dot(X2.T))**self.degree
        return res
    
    def rbf_kernel(self, X1, X2):
        X1 = np.atleast_2d(X1)
        X2 = np.atleast_2d(X2)
        res = np.exp(-self.gamma * dist.cdist(X1, X2)**2)
        return res
    
    def train(self, X, y):
        self.y=y
        if self.kernel == "linear":
            K = self.linear_kernel(X, X)
            kernel_func = self.linear_kernel
        elif self.kernel == "poly":
            assert isinstance(self.seta, int) and self.seta >= 0, "seta should be given as int and >= 0"
            assert isinstance(self.gamma, float) and self.gamma > 0, "gamma should be given as float and > 0"
            assert isinstance(self.degree, int) and self.degree >= 2, "degree should be given as int and >= 2"
            K = self.poly_kernel(X, X)
            kernel_func = self.poly_kernel
        elif self.kernel == "rbf":
            assert isinstance(self.gamma, float) and self.gamma > 0, "gamma should be given as float and > 0"
            K = self.rbf_kernel(X, X)
            kernel_func = self.rbf_kernel
        else:
            raise ValueError("kernel should be linear, poly or rbf")
        
        n = len(X)
        iters = 0
        a = np.zeros((1, n))[0]
        b = 0
        while iters < self.max_iters:
            iters += 1
            a_prev = copy.deepcopy(a)
            for i in range(n):
                Ki = kernel_func(X, X[i:i+1])
                ui = (a*y).T.dot(Ki) + b
                Ei = ui - y[i]
                # print(int(y[i]*Ei))
                if (int(y[i]*Ei) >= 1 and a[i] > 0) or (int(y[i]*Ei) <= 1 and a[i] < self.C) \
                    or (int(y[i]*Ei) == 1 and a[i] == 0) or (int(y[i]*Ei) == 1 and a[i] == self.C):
                    # Pick random i
                    j = self.random_idx(i,n)
                    # Error for i
                    Kj = kernel_func(X, X[j:j+1])
                    uj = (a*y).T.dot(Kj) + b # y_hat
                    Ej = uj - y[j]
                    # find_bounds
                    L, H = self.find_bounds(a, i, j)
                    # eta
                    
                    eta = K[i:i+1, i:i+1] + K[j:j+1,j:j+1] - 2*K[i:i+1,j:j+1]
                    eta_a=eta.any()<0
                    if eta_a.any() <= 0: continue
                    # Save old alphas
                    ai_old, aj_old = a[i], a[j]
                    # Update alpha
                    a[j] = aj_old + y[j]*(Ei-Ej)/eta
                    a[j] = self.clip(a[j], L, H)
                    a[i] = ai_old + y[j]/y[i]*(aj_old-a[j])
                    # Find intercept
                    b1 = b - Ei - y[i]*(a[i]-ai_old)*K[i,i] - y[j]*(a[j]-aj_old)*K[i,j]
                    b2 = b - Ej - y[i]*(a[i]-ai_old)*K[i,j] - y[j]*(a[j]-aj_old)*K[j,j]
                    if 0 < a[i] < self.C:
                        b = b1
                    elif 0 < a[j] < self.C:
                        b = b2
                    else:
                        b = 0.5 * (b1 + b2)
                else:
                    continue
            # Check convergence
            diff = np.sqrt(np.sum((a-a_prev)**2))
            if diff < self.tol:
                break
        return iters, kernel_func, a, b

- 测试类

In [10]:
class SvmPredict(object):
    def __init__(self, a, b, X, y, kernel_func):
        self.a = a
        self.b = b
        self.X =X
        self.y = y
        self.kernel_func = kernel_func
    
    def predict(self, X_predict):
        K = self.kernel_func(self.X, X_predict)
        y_hat = (self.a*self.y).T.dot(K) + self.b
        y_pred = np.sign(y_hat)
        return y_pred

## 训练模型

In [11]:
obj_train = SvmTrain(C=1, max_iters=100, tol=0.01, kernel="rbf", seta=None, gamma=0.5, degree=None)
iters, kernel_func, a, b = obj_train.train(data_train_x[:4000], data_train_y[:4000])

## 测试

- 测试集预测

In [14]:
try: 
    with open ("my_pre.txt","r+") as file:
        y_pred=np.array(eval(file.read()))
except FileNotFoundError:
    obj_predict = SvmPredict(a, b, data_train_x[:4000], data_train_y[:4000], kernel_func)
    y_pred = obj_predict.predict(data_test_x[:1000])
    with open("my_pre.txt","w+") as file:
        file.write(str(tuple(y_pred)))

- 计算正确率

In [15]:
print(sum(data_test_y[:1000] == y_pred)/len(data_test_y))

0.0098
